In [18]:
"""
Aashish Adhikari
"""


from __future__ import division
from __future__ import print_function

import sys
import time
try:
   import _pickle as pickle
except:
   import pickle

import numpy as np
import math


# This is a class for a LinearTransform layer which takes an input 
# weight matrix W and computes W x as the forward step



class LinearTransform(object):

    def __init__(self, W, b):
        self.first_layer_weights = W #weights for all n nodes in the hidden layer
        self.first_layer_bias = b
        self.relu_object = ReLU()
        self.sig = SigmoidCrossEntropy()
    
    
    def forward(self, x,y):
        #print("aa",x.shape,self.weights.shape)
        #Never do this because it does element-wise and this gives an error
        #batch_linear_summation = x * self.weights# + self.bias
        
        
        batch_linear_summation_without_relu = np.dot(x, self.weights) + self.bias
        #print("batch_linear_summation is ",batch_linear_summation)
        return (batch_linear_summation_without_relu,self.relu_object.forward(batch_linear_summation_without_relu,y))
        
    def forward_2(self,x):
        batch_linear_summation_without_sigmoid = np.dot(x,self.weights)+self.bias
        return (batch_linear_summation_without_sigmoid,self.sig.forward(batch_linear_summation_without_sigmoid))
        print()
    def backward(
        self, 
        grad_output, 
        learning_rate=0.0, 
        momentum=0.0, 
        l2_penalty=0.0,
    ):
        print()
	# DEFINE backward function
# ADD other operations in LinearTransform if needed



# This is a class for a ReLU layer max(x,0)



class ReLU(object):
    def __init__(self):
        print()
        
    def forward(self, x,y):
	# DEFINE forward function
        relu_output = np.maximum(0,x)
        #print("relu_output is",relu_output)
        return relu_output
        
           
    def backward(
        self, 
        grad_output, 
        learning_rate=0.0, 
        momentum=0.0, 
        l2_penalty=0.0,
    ):
        print()
    # DEFINE backward function
# ADD other operations in ReLU if needed



# This is a class for a sigmoid layer followed by a cross entropy layer, the reason 
# this is put into a single layer is because it has a simple gradient form



class SigmoidCrossEntropy(object):
    def __init__(self):
        print()
        #self.linear_transform_object_2 = LinearTransform(self.second_layer_weights,self.second_layer_bias)
        
    def forward(self, x):
#         linear_summation = self.linear_transform_object_2.forward(self,x)
#         print("linear summation size ",linear_summation.shape)
#         #linear transform gar paile
        #ani balla sigmoid
        
        
        
        #print("sigmoid ma aayeko input ", x)
        
        sigmoid =  (1 / (1 + np.exp(-x)))
        #print("sigmoid is ", sigmoid)
        return sigmoid
        
    def backward(self,second_layer_wts, learning_rate,direction, momentum, l2_penalty, grad_output, second_layer_bias, grad_bias):
        
        print("dimensions ",second_layer_wts,learning_rate,direction,momentum,l2_penalty,grad_output)
        direction_updated = momentum * direction - np.dot(learning_rate, ( grad_output + np.dot(l2_penalty, second_layer_wts )))
        print("direction updated orr weight update is ",direction_updated)
        return(direction_updated, second_layer_wts +  direction_updated )
        
        
        
        
        
        print()




# This is a class for the Multilayer perceptron




class MLP(object):

    def __init__(self, input_dims, hidden_units):
    # INSERT CODE for initializing the network
        self.input_dims = input_dims
        self.hidden_units = hidden_units
        
        
        
        self.first_layer_weights = np.random.rand(input_dims, hidden_units) #% (0.2 + 1 - 0) + 0
        self.first_layer_biases = np.random.rand(hidden_units) #%  (0.2 + 1 - 0) + 0
        self.second_layer_weights = np.random.rand(hidden_units,1) #%  (0.2 + 1 - 0) + 0
        self.second_layer_bias = np.random.rand(1) #%  (0.2 + 1 - 0) + 0
        
        
        
        self.sigmoid_object_for_backpass = SigmoidCrossEntropy()
        
        #------------------------------------
        self.x_for_this_batch = []
        self.y_for_this_batch = []
        self.y_cap_for_this_batch = []
        
        
        self.hidden_layer_output = [] #Relu through aayeko xa sigmoid of A1
        self.sigmoid_output = []#Sigmoid through aayeko is Relu of A2, equals y cap for this batch
        
        self.hidden_layer_op_before_Relu = [] #A1
        self.sigmoid_layer_op_before_sigmoid = []#A2
        #------------------
        
        
        
        
#         #gradients
#         self.dE_by_d_y_cap = np.array(10)
        
       
        #first hidden layer ko lagi linear transform
        self.linear_transform_object = LinearTransform(self.first_layer_weights,self.first_layer_biases)#remember the dilemma you got here,
        #aafule pass garnu parne bhayepaxi jasati ni afnai attributes hunxa ni ta, so u never face that dilemma
        
        
        #second layer = op layer ko lagi linear transform
        self.linear_transform_object_for_sigmoid = LinearTransform(self.second_layer_weights,self.second_layer_bias)
        
        
        
        
    def train(self,x_batch,y_batch,learning_rate,momentum,l2_penalty):
        
        self.x_for_this_batch = x_batch
        self.y_for_this_batch = y_batch
        #print("within train x_batch, y_batch",x_batch,y_batch)
        #print("x batch and y batch ",len(x_batch),len(y_batch))
        
        
        
        
        
        
        
        self.hidden_layer_op_before_Relu, self.hidden_layer_output = self.linear_transform_object.forward(x_batch,y_batch)
        #print("within train hiden layer output",self.hidden_layer_output)
        
        
        #self.sigmoid_layer_op_before_sigmoid, self.sigmoid_output = self.linear_transform_object_for_sigmoid.forward_2(self.hidden_layer_output)
        #dont set this sigmoid op to global, has 1
        
        
        
        self.sigmoid_layer_op_before_sigmoid, sigmoid_output = self.linear_transform_object_for_sigmoid.forward_2(self.hidden_layer_output)
        
        
        #avoid overflow and underflow
        
        self.sigmoid_output = np.where(sigmoid_output == 0,0.00000001,sigmoid_output)
        
        
        self.sigmoid_output = np.where(sigmoid_output == 1,1-0.00000001,sigmoid_output)

        
        
        
        print("within train sigmoid  output",sigmoid_output)
        
        
        
        return self.sigmoid_output
        
        #yeha bata linear transform ma pathai
            
            
        #10 ota palai palo ayepaxi tiotal cumulative ma kaam garne yeha
        
            
        
        ######Yeha k garne bhar aaba
        
        
	# INSERT CODE for training the network

    
    
    
    
    def evaluate(self, y_cap, y):
        print("y cap and y for loss ", y_cap, y)
        y = np.transpose(y)
#         loss = 

#         print (" y is ",y)
#         print("y cap is",y_cap)
        
        #if (y_cap)
        

        #avoid overflow and underflow
        
        y_cap = np.where(y_cap == 0,0.00000001,y_cap)
        
        
        y_cap = np.where(y_cap == 1,1-0.00000001,y_cap)

         
        loss = -(np.dot(y,np.log(y_cap)) + np.dot((1-y),(np.log(1-y_cap))))
        
        
        #now the loss is 1-dimensional
        print("loss is  ",loss) 
        
        return loss
        
            
        
        #do weight update for all layers from here
    def weight_update(self, loss_for_the_batch, lr, direction_passed, momentum,l2_penalty):
        
#         ########
#         self.input_dims = input_dims
#         self.hidden_units = hidden_units
#         self.first_layer_weights = np.random.rand(input_dims, hidden_units) #% (0.2 + 1 - 0) + 0
#         print("first_layer_weights ",self.first_layer_weights)
#         self.first_layer_biases = np.random.rand(hidden_units) #%  (0.2 + 1 - 0) + 0
#         self.second_layer_weights = np.random.rand(hidden_units,1) #%  (0.2 + 1 - 0) + 0
#         self.second_layer_bias = np.random.rand(1) #%  (0.2 + 1 - 0) + 0
        
        
#         #------------------------------------
#         x_for_this_batch = []
#         y_for_this_batch = []
#         self.hidden_layer_output = [] #Relu through aayeko xa sigmoid of A1 Z1
        
#         self.sigmoid_output = []#Sigmoid through aayeko is Relu of A2 Z2
        
#         self.hidden_layer_op_before_Relu = [] #A1
#         self.sigmoid_layer_op_before_sigmoid = []#A2
#         #######
        
    
        print("y truth",self.y_for_this_batch,"y cap",self.y_cap_for_this_batch)
        print("")
        
        
#         p = np.array([[2,2],[3,3]])
#         q = np.array([[2,2],[3,3]])
#         print(" p by q", p/q)
#         print(" p divide q", np.divide(p,q))
#         time.sleep(100)

        
         
        
        x1 = np.dot(self.y_for_this_batch, np.transpose(( 1- self.y_for_this_batch)))
        x2 = (np.dot((self.y_cap_for_this_batch),np.transpose((1-self.y_cap_for_this_batch))))
        a = x1/x2
        
        b = np.dot(self.sigmoid_output, np.transpose((1 - self.sigmoid_output)))
        
        
        
        dE_by_dW2 = np.dot( np.dot(a,b),self.hidden_layer_output)
        
        
        
        
#         print("size ",x1.size,x2.size,a.size,b.size,dE_by_db2.size,dE_by_dW2.size)
        
        
        
#         print("a, b, grad ",a,b,dE_by_db2, self.second_layer_bias)
        
        #The dimension of dE by dW2 is 10 by 2
        #10 examples taken in a batch so each of them give a unique feedback for both the weights in W2 for the 2 input weights
        #to the sigmoid node
        #we need to sum them up and take an average to do the update
        
        dE_by_dW2 = (dE_by_dW2.sum(axis = 0))/10#--------------------------------dont forget to change batch size here
        
        
        
        
        
        
        #call the sigmoid unit to do the update on the weights by passing the necessary arguments
        
        
        #second layer weights updated here
        direction_passed, self.second_layer_weights =  self.sigmoid_object_for_backpass.backward(self.second_layer_weights, lr, direction_passed, momentum,l2_penalty, dE_by_dW2)
        
        #------------------second layer bias update
        
        
        dE_by_db2 = np.dot(a,b)
        
        self.second_layer_bias = self.
        
        
        
        
        #time.sleep(5)
        #NOW UPdate the first ;layer weights
        
        
        #dE_by_d_y_cap = y - 
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
	# INSERT CODE for testing the network
# ADD other operations and data entries in MLP if needed









if __name__ == '__main__':
    if sys.version_info[0] < 3:
        print("system version is less than 3") 
        data = pickle.load(open('dataset_folder/cifar_2class_py2.p', 'rb'))
        
    else:
        #train_x, train_y, test_x, test_y 
        data = pickle.load(open('../../dataset_folder/cifar_2class_py2.p', 'rb'), encoding='bytes')
        
    #print(data)
    #print(data[b'test_data'])
    
    
    train_x = np.array(data[b'train_data'])
    train_y = np.array(data[b'train_labels'])
    test_x = np.array(data[b'test_data'])
    test_y = np.array(data[b'test_labels'])
    
    
    
    def normalize(x):
         
        top = x - np.amin(x)
        bottom = np.amax(x)-np.amin(x)
        return(top/bottom)
    
    
    #for checking------------------remove paxi
    #print("dtype is ",train_x.dtype)
    train_x = train_x[...,0:1000]
    train_y = train_y[...,0:1000]
    test_x = test_x[...,0:1000]
    test_y = test_y[...,0:1000]
    
    
    
    #print(" minimum in x is ", np.amax(train_x))
#     print(" maximum in x is ", np.amax(train_x))
#     train_x = normalize(train_x)
#     print(" normalized x is ",train_x)
#     print(" maximum in x is ", np.amax(train_x))

    
    
    

    num_examples, input_dims = train_x.shape
    
    
	# INSERT YOUR CODE HERE
    
    
    #ask the user about the the number of hidden nodes hs/ she wants
    
    
    #inp = input("How many hidden nodes do you want?\n")
    #num_of_hidden_nodes = int(inp)
    
    
    num_of_hidden_nodes = 2
#     print("dimension of each example is ",input_dims)
    
    
	
    
    
    # YOU CAN CHANGE num_epochs AND num_batches TO YOUR DESIRED VALUES
    
    
    num_epochs = 10
    num_batches = 1000
    learning_rate = [0.001,0.003,0.01,0.03,0.1,0.3,1,3,10]
    inertia_of_momentum = [0.001,0.003,0.01,0.03,0.1,0.3,1,3,10]
    l2_penalty_factor = [0.0000001,0.0000003,0.000001,0.000003,0.00001,0.00003,0.0001,0.0003,0.001,0.003,0.01,0.03,1,3,10]
    print("Choose the corresponding index number for the learning rate you want to use")
    #lr = int(input("[0.001,0.003,0.01,0.03,0.1,0.3,1,3,10]"))
    lr =5 
    print("Choose the corresponding index number for the inertia of momentum you want to use")
    #iner = int(input("[0.001,0.003,0.01,0.03,0.1,0.3,1,3,10]"))
    iner = 1
    print("Choose the corresponding index number for the L2 penalty factor you want to use")
    #penalty = int(input("[0.0000001,0.0000003,0.000001,0.000003,0.00001,0.00003,0.0001,0.0003,0.001,0.003,0.01,0.03,1,3,10]"))
    penalty = 1
    mlp = MLP(input_dims, num_of_hidden_nodes)
    
    
    
    epoch_num = 1
    for epoch in range(num_epochs):
        print(" Epoch is ",epoch_num)

	# INSERT YOUR CODE FOR EACH EPOCH HERE
        total_loss_for_epoch = 0.0
        direction = 0
        for b in range(num_batches):
            
            
            
            #print("num of examples num of batches ",num_examples, " ",num_batches)
            
            
            
            batch_start = int( (num_examples / num_batches) * b)
            batch_end = int((num_examples / num_batches)*(b +1))
            
            
            
#             print("batch start and end",batch_start," ",batch_end)
#             print("Chosen learning rate, inertia of momentum and l2 penalty factor are")
            #print(learning_rate[lr], inertia_of_momentum[iner], l2_penalty_factor[penalty])
            print("train size is  ",train_x.shape)
            
            
            
            
            
            mlp.x_for_this_batch = train_x[batch_start:batch_end,...]
            mlp.y_for_this_batch = train_y[batch_start:batch_end,...]
            mlp.y_cap_for_this_batch = mlp.train(mlp.x_for_this_batch,mlp.y_for_this_batch,int(learning_rate[lr]), inertia_of_momentum[iner], l2_penalty_factor[penalty])
            
            
            
            
            
            #print("y_cap_for_batch",mlp.y_cap_for_this_batch)
            
            loss = mlp.evaluate(mlp.y_cap_for_this_batch,mlp.y_for_this_batch )
            
            
            
            total_loss_for_epoch = total_loss_for_epoch + loss
            
            #call for weight updates,  need to pass ionly the loss
            
            mlp.weight_update(loss, learning_rate[lr], direction, inertia_of_momentum[iner], l2_penalty_factor[penalty])
            
            
            
            
            
            
            
            
            
            
            # INSERT YOUR CODE FOR EACH MINI_BATCH HERE
            # MAKE SURE TO UPDATE total_loss
            
            
            
            #print('\r[Epoch {}, mb {}]    Avg.Loss = {:.3f}'.format(epoch + 1,b + 1,total_loss_for_epoch,),end='', )
            
            #after each mini bach update you want to update the momentum value
            sys.stdout.flush()
            # INSERT YOUR CODE AFTER ALL MINI_BATCHES HERE
        epoch_num +=1
        
        direction = 0
        # MAKE SURE TO COMPUTE train_loss, train_accuracy, test_loss, test_accuracy
    
        
        print()
        #do for each epoch this 
        
        
        print('    Train Loss: {:.3f}    Train Acc.: {:.2f}%'.format(
            train_loss,
            100. * train_accuracy,
        ))
        print('    Test Loss:  {:.3f}    Test Acc.:  {:.2f}%'.format(
            test_loss,
            100. * test_accuracy,
        ))


Choose the corresponding index number for the learning rate you want to use
Choose the corresponding index number for the inertia of momentum you want to use
Choose the corresponding index number for the L2 penalty factor you want to use





 Epoch is  1
train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]] [[1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]]
loss is   [[110.52408447]]
y truth [[1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]] y cap [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]]

dimensions  [[0.41601434]
 [0.63437315]] 0.3 0 0.003 3e-07 [1484667.68409486 1551376.29178584]
direction updated orr weight update is  [[-445400.3052285  -465412.88753579]
 [-4

train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]] [[1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]]
loss is   [[73.68272302]]
y truth [[1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]] y cap [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]]

dimensions  [[-4402211.19723958 -4590986.81202702]
 [-4402210.97888097 -4590986.5936684 ]] 0.3 0 0.003 3e-07 [1411336.15095079 1465227.63159951]
direction updated orr weight update is  [[-423400.44908623 -439567.87629104]
 [-423400.44908625 -439567.87629106]]
train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0

train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]] [[0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]]
loss is   [[73.68272302]]
y truth [[0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]] y cap [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]]

dimensions  [[-8815187.34247979 -9184756.83735413]
 [-8815187.12412138 -9184756.61899571]] 0.3 0 0.003 3e-07 [1370561.55094879 1422935.17185289]
direction updated orr weight update is  [[-411167.67191778 -426879.72492775]
 [-411167.67191779 -426879.72492777]]
train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0

train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]] [[0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]]
loss is   [[110.52408447]]
y truth [[0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]] y cap [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]]

dimensions  [[-13178145.98572334 -13724963.43654595]
 [-13178145.76736512 -13724963.21818772]] 0.3 0 0.003 3e-07 [1389131.64982295 1442514.74393389]
direction updated orr weight update is  [[-416738.30891375 -432753.18793346]
 [-416738.30891377 -432753.18793348]]
train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999

train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]] [[0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]]
loss is   [[110.52408447]]
y truth [[0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]] y cap [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]]

dimensions  [[-17687717.42472346 -18421851.95050064]
 [-17687717.20636544 -18421851.73214261]] 0.3 0 0.003 3e-07 [1652021.28125693 1716659.74129811]
direction updated orr weight update is  [[-495604.79248251 -514996.26442276]
 [-495604.79248253 -514996.26442278]]
train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999

...

within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]] [[1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]
loss is   [[110.52408447]]
y truth [[1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]] y cap [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]]

dimensions  [[-4.10977248e+08 -4.27807954e+08]
 [-4.10977248e+08 -4.27807954e+08]] 0.3 0 0.003 3e-07 [1650931.21683939 1716833.18962054]
direction updated orr weight update is  [[-495242.37709947 -515011.45417027]
 [-495242.37709949 -515011.45417029]]
train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999

train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]] [[1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]]
loss is   [[92.10340374]]
y truth [[1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]] y cap [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]]

dimensions  [[-4.16055441e+08 -4.33097535e+08]
 [-4.16055440e+08 -4.33097535e+08]] 0.3 0 0.003 3e-07 [1640493.84559521 1703562.36202032]
direction updated orr weight update is  [[-492110.7086889  -511029.72982795]
 [-492110.70868892 -511029.72982797]]
train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.9999999

train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]] [[1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]]
loss is   [[92.10340374]]
y truth [[1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]] y cap [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]]

dimensions  [[-4.20924655e+08 -4.38162679e+08]
 [-4.20924654e+08 -4.38162678e+08]] 0.3 0 0.003 3e-07 [1746163.85828353 1828257.56747996]
direction updated orr weight update is  [[-523811.27426614 -548437.83560291]
 [-523811.27426616 -548437.83560293]]
train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.9999999

train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]] [[0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]]
loss is   [[73.68272302]]
y truth [[0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]] y cap [[0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]
 [0.99999999]]

dimensions  [[-4.25787531e+08 -4.43229466e+08]
 [-4.25787531e+08 -4.43229466e+08]] 0.3 0 0.003 3e-07 [1522214.24401325 1584264.54091675]
direction updated orr weight update is  [[-456625.95232621 -475239.47162307]
 [-456625.95232623 -475239.47162309]]
train size is   (10000, 1000)
within train sigmoid  output [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
y cap and y for loss  [[0.99999999]
 [0.99999999]
 [0.9999999

NameError: name 'train_loss' is not defined